In [1]:
import pandas as pd
import numpy as np
import time
from tqdm import tqdm #https://github.com/tqdm/tqdm

import warnings
warnings.filterwarnings("ignore")
import gc; gc.enable()

In [2]:
# from LGBWrapper import RegressorModel
# from LGBWrapper import LGBWrapper_regr
# from sklearn.model_selection import KFold, GroupKFold

from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import ElasticNetCV, LassoCV, RidgeCV
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.svm import SVR
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import AdaBoostClassifier

from sklearn.metrics import cohen_kappa_score
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor

In [3]:
#Display setting to see everything in each row
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 600)

In [4]:
gc.collect()

40

# I) Import data

In [5]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type
        to reduce memory usage.
    """
    start_mem = df.memory_usage().sum() / 1024**2
    
    for col in df.columns:
        col_type = df[col].dtype
        
        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
        #else:
            #df[col] = df[col].astype('category')

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB --> {:.2f} MB (Decreased by {:.1f}%)'.format(
        start_mem, end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [9]:
%%time
train=pd.read_csv('input/train.csv')
train=reduce_mem_usage(train)
train.timestamp=pd.to_datetime(train['timestamp'])
train=train.sort_values(by=['installation_id','timestamp'])

test=pd.read_csv('input/test.csv')
test=reduce_mem_usage(test)
test.timestamp=pd.to_datetime(test['timestamp'])
test=test.sort_values(by=['installation_id','timestamp'])

NameError: name 'reduce_mem_usage' is not defined

In [8]:
train_labels=pd.read_csv("input/train_labels.csv")
specs=pd.read_csv("input/specs.csv")

In [12]:
test.shape

(1156414, 11)

# II) Preprocess train and train labels


**1) Preprocessing train**

Function that deletes all the rows after the last assessment for each installation_id

In [13]:
import itertools

def remove_rows_after_last_assignment(df): #remember to pass in train
    #CREAT A LIST OF INDEX OF LAST ASSIGNMENT AND FIRST ELEMENT FOR EACH PLAYER
    df['index']=df.index
    index_lastAsessment_keep=df[df.type=='Assessment'].groupby('installation_id',as_index=False).last()
    index_first_keep=df.groupby('installation_id',as_index=False).first()

    index_lastAsessment_keep=np.array(index_lastAsessment_keep['index'])
    index_first_keep=np.array(index_first_keep['index'])
    
    #CREAT A COLUMN THAT HAS 1 FOR DELETION AND 0 FOR KEEP
    index_first_remove=index_lastAsessment_keep+1
    index_lastAsessment_remove=index_first_keep-1
    index_lastAsessment_remove=np.delete(index_lastAsessment_remove,0)
    index_lastAsessment_remove=np.append(index_lastAsessment_remove,len(train)-1)

    interval_keep=index_lastAsessment_keep-index_first_keep+1
    interval_delete=index_lastAsessment_remove-index_first_remove+1
    
    mark4del=[]
    for i in range(len(interval_keep)):
        mark4del.append(np.zeros(interval_keep[i]))
        mark4del.append(np.ones(interval_delete[i]))
    
    
    mark4del = list(itertools.chain.from_iterable(mark4del))
    #ADD THE NEW COLUMN
    df['mark4del']=mark4del
    df=df[df.mark4del==0]
    df=df.drop(['index', 'mark4del'],axis=1)
    return df

Extract installation_id that is not in the train_labels data set. These id has default accuracy group 0

In [14]:
%%time
#Find people who has no labels but took the assessment
no_record=train[~train.installation_id.isin(train_labels.installation_id)]
id_no_record_with_assessment=no_record[no_record.type=='Assessment'].installation_id.unique()

#...and add them to the train set
installation_id_use=np.concatenate((train_labels['installation_id'].unique(),id_no_record_with_assessment),
                                   axis=None)
train=train[train.installation_id.isin(installation_id_use)]

#Remove all activities after last assessment
train=train.sort_values(by=['installation_id','timestamp']).reset_index(drop=True)
train=remove_rows_after_last_assignment(train)

#Delete Non-type world
train=train[train.world!='NONE']

CPU times: user 20.4 s, sys: 10.3 s, total: 30.7 s
Wall time: 34.6 s


In [15]:
len(installation_id_use)

4242

So lenght of train change from 11m -> 8294138 (after extract the one with lables) -> 7019762 (after delete the unused rows) -> 7012808 (after delete world NONE)

**2) Preprocessing labels**

Need also add the new data to the train_labels data sets

In [16]:
%%time
#We are going to add new rows (the players who never finish an assessment) 
# and new columns (world, game, time) to the train_labels
temp=train[((train.type=='Assessment') 
            & train.game_session.isin(train_labels.game_session))].groupby('game_session', as_index=False).last()

temp = temp[['game_session', 'event_count', 'game_time', 'world' ]]

train_labels=pd.merge(train_labels, temp, on='game_session').reset_index(drop=True)

## Add timestamp to train session
temp=train[train.game_session.isin(train_labels.game_session) & 
      (train.type=='Assessment')].groupby("game_session", as_index=False).last()[['game_session','timestamp']]

train_labels=pd.merge(train_labels, temp, on='game_session').reset_index(drop=True)

#Add  the extra information we found 
list_assessment_no_record=no_record[no_record.type=='Assessment'].groupby("game_session", as_index=False).last()
list_assessment_no_record['num_correct']=0
list_assessment_no_record['num_incorrect']=0
list_assessment_no_record['accuracy']=0
list_assessment_no_record['accuracy_group']=0
list_assessment_no_record=list_assessment_no_record[train_labels.columns]

train_labels=pd.concat([train_labels,list_assessment_no_record])
train_labels=train_labels.sort_values(by=['installation_id','timestamp']).reset_index(drop=True)

CPU times: user 3.17 s, sys: 290 ms, total: 3.46 s
Wall time: 3.3 s


In [17]:
train_labels.head()

,game_session,installation_id,title,num_correct,num_incorrect,accuracy,accuracy_group,event_count,game_time,world,timestamp
0,901acc108f55a5a1,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3,48,39803,TREETOPCITY,2019-08-06 05:22:41.147000+00:00
1,77b8ee947eb84b4e,0006a69f,Bird Measurer (Assessment),0,11,0.0,0,87,92799,TREETOPCITY,2019-08-06 05:36:51.915000+00:00
2,6bdf9623adc94d89,0006a69f,Mushroom Sorter (Assessment),1,0,1.0,3,35,26827,TREETOPCITY,2019-08-06 05:38:16.835000+00:00
3,9501794defd84e4d,0006a69f,Mushroom Sorter (Assessment),1,1,0.5,2,42,31843,TREETOPCITY,2019-08-06 20:35:25.648000+00:00
4,a9ef3ecb3d1acc6a,0006a69f,Bird Measurer (Assessment),1,0,1.0,3,32,36368,TREETOPCITY,2019-08-06 20:50:35.426000+00:00


The information that we are not using are those session that is after the last assessment. This is because we are those extra information has no contribution to the performance of the last assessment. The following function remove it all

# III) Feature enginneer

### 1) Groupby session

In [18]:
def groupby_session(df):
    f=lambda x: (x.event_id.value_counts(), x.timestamp.values[0], x.event_count.max(), 
                 x.event_code.value_counts(), x.game_time.max(),
                 x.title.values[0], x.type.values[0], x.world.values[0], x.installation_id.values[0])

    temp=df.groupby("game_session").apply(f)
    
    df_new=pd.DataFrame({})
    df_new['installation_id']=[i[8] for i in temp.values]
    df_new["game_session"]=temp.index

    #Event_id
    for event_id in train.event_id.unique():
        df_new[event_id]=[i[0][event_id] if (event_id in i[0].index) else 0 for i in temp.values ]

    df_new['first_timestamp']=[i[1] for i in temp.values]
    df_new['event_count']=[i[2] for i in temp.values]
    df_new['game_time']=[i[4] for i in temp.values]
    df_new['title']=[i[5] for i in temp.values]
    df_new['type']=[i[6] for i in temp.values]
    df_new['world']=[i[7] for i in temp.values]

    #OHE event
    for event_code in train.event_code.unique():
        df_new[event_code]=[i[3][event_code] if (event_code in i[3].index) else 0 for i in temp.values ]

    df_new=df_new.sort_values(by=['installation_id', 'first_timestamp']).reset_index(drop=True)
    
    # Marking all assessments that have label
    #make_for_group
    if '00abaee7' in list(df_new.installation_id): #First iid found in test
        index_group=list(np.array(df_new[df_new.type=='Assessment'].index)+1)
    else:
        index_group=list(np.array(df_new[df_new.game_session.isin(train_labels.game_session)].index)+1)
 
    index_group_off=index_group[:-1]
    index_group_off.insert(0,0)
    index_diff=np.array(index_group)-np.array(index_group_off)

    temp=[np.ones(i)*index for index, i in enumerate(index_diff)]
    temp=np.concatenate(temp, axis=0)
    df_new['make_for_group']=temp.astype(int)
    
    return df_new

### 2) Groupby assignment

**Numerical and categorical**

In [19]:
def numerical_categorical_extraction(df):

    ##########Numerical##########
    df_=df[['make_for_group','world', 'game_time']].groupby(["make_for_group","world"], as_index=False).sum()
    
    #Find gametime for each world
    df_numerical=pd.DataFrame({})
    for element in df_.world.unique():
        index_has_values=df_[df_.world==element].make_for_group.values
        game_time_element=df_[df_.world==element].game_time
        temp=np.zeros(df_.make_for_group.nunique())
        temp[index_has_values]=game_time_element
        df_numerical["GT_world_"+element]=temp
    
    #Find gametime for each title
    df_=df[['make_for_group','title', 'game_time']].groupby(["make_for_group","title"], as_index=False).sum()

    for element in df_.title.unique():
        index_has_values=df_[df_.title==element].make_for_group.values
        game_time_element=df_[df_.title==element].game_time
        temp=np.zeros(df_.make_for_group.nunique())
        temp[index_has_values]=game_time_element
        df_numerical["GT_title_"+element]=temp
        
    #Find gametime for each type
    df_=df[['make_for_group','type', 'game_time']].groupby(["make_for_group","type"], as_index=False).sum()

    for element in df_.type.unique():
        index_has_values=df_[df_.type==element].make_for_group.values
        game_time_element=df_[df_.type==element].game_time
        temp=np.zeros(df_.make_for_group.nunique())
        temp[index_has_values]=game_time_element
        df_numerical["GT_type_"+ element]=temp
    
    #All the leftovers numerical

    df_=df.groupby(["make_for_group"], as_index=False).sum()
    df_numerical[df_.columns]=df_
    
    ##########Categrocial##########
    
    f= lambda x: (x.installation_id.unique()[0], len(x.game_session), 
              x.first_timestamp.values[0],  x[x.type!='Assessment'].title.value_counts(),
              x.title.values[-1], x.world.value_counts())
    categorical=df.groupby("make_for_group", as_index=False).apply(f)
    
    df_categorical=pd.DataFrame({})
    df_categorical['installation_id']=[i[0] for i in categorical.values]
    df_categorical['num_game_session']=[i[1] for i in categorical.values]
    df_categorical['first_timestamp']=[i[2] for i in categorical.values]

    for title in train[train.type!='Assessment'].title.unique():
        df_categorical['TITLE_'+title]=[i[3][title] if (title in i[3].index) else 0 for i in categorical.values ]

    df_categorical['Assessment_title']=[i[4] for i in categorical.values]

    for world in df.world.unique():
        df_categorical["WORLD_"+title]=[i[5][world] if (world in i[5].index) else 0 for i in categorical.values ]
    
    #Game_session
    df_categorical['game_session']=df.groupby("make_for_group", as_index=False).last().game_session
    df=pd.concat([df_numerical, df_categorical],axis=1)
    
    return df

### 3) Adding time_feature 

Approximation time spend from one assessment to the other

In [20]:
def time_feature(df):
    def time_distance_func(df):
        time=df.first_timestamp
        time_off=df.first_timestamp.shift(1, fill_value=0)
        result=list(time-time_off)
        result[0]=0
        return result

    #Dummy title
    temp=pd.get_dummies(df.Assessment_title, prefix='Record_')
    df=pd.concat([df, temp], axis=1)
    
    df_temp=df[['first_timestamp', 'installation_id']]
    
    #Convert timestamp to integer
    df['first_timestamp']=df['first_timestamp'].astype('int64')//1e6
    
    #Calculate the time interval playing one assignment to the other
    temp=df.groupby('installation_id').apply(time_distance_func).values
    temp=np.concatenate(temp)

    df['time_distance_assessment']=temp
    #installation_id label
    list_iid=df['installation_id'].value_counts()
    list_iid=pd.DataFrame({'installation_id': list_iid.index, 
                           'count': list_iid.values}).sort_values('installation_id')
    temp=[np.ones(i)*index for index, i in enumerate(list_iid['count'].values)]
    temp=np.concatenate(temp, axis=0)
    df['iid_Label']=temp.astype(int)
    df=df.sort_values(by=['installation_id', 'first_timestamp']).reset_index(drop=True)
    return df

### 4) Adding leakage

In [21]:
def accuracy_group_feature(df, is_train=True):
    def event_data(x):
        if (x.title.unique()[0]=='Bird Measurer (Assessment)'):
            temp=x[x.event_code==4110]
            true=temp.event_data.str.contains('true').sum()
            false=temp.event_data.str.contains('false').sum()
            return (true,false)
        else:
            temp=x[x.event_code==4100]
            true=temp.event_data.str.contains('true').sum()
            false=temp.event_data.str.contains('false').sum()
            return (true,false)
        
    def extract_game_correction(df):
        if df.game_session.isin(train.game_session).all():
            temp=train[train.game_session.isin(df.game_session)]
            result=temp.groupby('game_session').apply(event_data)
            return result
        elif df.game_session.isin(test.game_session).all():
            temp=test[test.game_session.isin(df.game_session)]
            result=temp.groupby('game_session').apply(event_data)
            return result
        else:
            print('This is invalid')

    
    leak_info=extract_game_correction(df)
    leak_info_true=np.array([i[0] for i in leak_info.values])
    leak_info_false=np.array([i[1] for i in leak_info.values])

    total=[i if i!=0 else 1 for i in leak_info_true+leak_info_false] #Avoid dividing by 0
    accuracy = leak_info_true/total 
    
    
    df_temp=pd.DataFrame({})
    game_session_temp=leak_info.index
    df_temp['game_session']=game_session_temp
    df_temp['acc_num_correct']=leak_info_true
    df_temp['acc_num_incorrect']=leak_info_false
    df_temp['acc_accuracy_group']=accuracy
    
    df=pd.merge(df_temp, df, on='game_session')
    df=df.sort_values(by=['installation_id', 'first_timestamp']).reset_index(drop=True)
    return df
    

### 5) Cum sum

#### Cumsum

In [22]:
#Function to detect categorical column
def cat_col(df):
    cat_col_list=[]
    for col in df.columns:
        if (df[col].dtype=='O'):
            cat_col_list.append(col)
    return cat_col_list

#CUMSUM
def cumsum(df):
    #cumsum selected cols and keep the rest
    all_cols=set(df.columns)
    excluded_columns=set(cat_col(df)).union({'iid_Label'})
#     other=set(['acc_num_correct','acc_num_incorrect', 'acc_accuracy_group', 'iid_Label'])
    df_new=df.groupby('installation_id').cumsum() #!!!!
    
    for col in excluded_columns:
        df_new[col]=df[col]
    
    #some extra feature engineering
    temp=pd.get_dummies(df_new.Assessment_title, prefix='ASM_')
    df_new=pd.concat([df_new, temp], axis=1)
    
    #game on that session
    temp=train[train.game_session.isin(df_new.game_session)]
    if len(temp)==0:
        temp=test[test.game_session.isin(df_new.game_session)]
        
    ASM_GT=temp.groupby('game_session').last().game_time.values
    
    df_new['ASM_GT']=ASM_GT
    
    #Average gametime per session
    df_new['Average_GT_per_session']=df.game_time/df.num_game_session
    
    #########IMPORTANT##########
    df_new['ASM_correct']=df['acc_num_correct']
    df_new['ASM_incorrect']=df['acc_num_incorrect']
    df_new['ASM_accuracy']= df['acc_accuracy_group']
     
    ASM_num_assess=(df_new['Record__Bird Measurer (Assessment)'] + df_new['Record__Cart Balancer (Assessment)']+
                df_new['Record__Cauldron Filler (Assessment)']+ df_new['Record__Chest Sorter (Assessment)']
                +df_new['Record__Mushroom Sorter (Assessment)'])
            
    df_new['ASM_num_assess']=ASM_num_assess
    df_new['Average_GT_assess']=df_new['GT_type_Assessment']/df_new['ASM_num_assess']
    
    #acc_accuracy_group
    total=[i if i!=0 else 1 for i in list(df_new['acc_num_correct']+df_new['acc_num_incorrect'])] #Avoid dividing by 0
    accuracy = df_new['acc_num_correct']/total 
    df_new['acc_accuracy_group']=accuracy
    ############################
    #Delete make for group
    df_new=df_new.drop('make_for_group', axis=1)
    
    
    #sort_function
    all_cols=df_new.columns
    ec_cols=list(train.event_code.unique())
    ei_cols=list(train.event_id.unique())
    others=list(set(all_cols)-set(ec_cols).union(set(ei_cols)))
    others.sort()
    df_new=df_new[others+ec_cols+ei_cols]
    
    return df_new


### 6) Main transformer

In [23]:
first1k=train.installation_id.unique()[:50]
sample=train[train.installation_id.isin(first1k)]

In [24]:
def main(df):
    def convert(seconds): 
        return time.strftime("%H:%M:%S", time.gmtime(seconds)) 

    start=time.time()
    print("Starting to process the information .......")
    df=groupby_session(df)
    print("Finish step 1, time elapsed: ", convert((time.time()-start)))
    
    df=numerical_categorical_extraction(df)
    print("Finish step 2, time elapsed: ", convert((time.time()-start)))
    
    df=time_feature(df)
    print("Finish step 3, time elapsed: ", convert((time.time()-start)))
    
    df=accuracy_group_feature(df)
    print("Finish step 4, time elapsed: ", convert((time.time()-start)))
    
    df=cumsum(df)
    print("Finish step 5, time elapsed: ", convert((time.time()-start)))
    
    return df

In [25]:
def groupby_iid(df):
    df_new=df.groupby('installation_id', as_index=False).last()
#     df_new['Average_GT_per_session']=df.game_time/df.num_game_session
    return df_new

In [28]:
def check_labels_in_train_labels(df):
    temp=label(df)
    temp=pd.merge(train_labels, temp, on='game_session')
    error= sum(temp.accuracy_group==temp.group)
    print( "Num correct labels are:", error )
    print( "Percentage of correctness:", round(error*100/len(df),2))
    
def label(df):
    group=[]
    for i in df.ASM_accuracy:
        if i==0:
            group.append(0)
        elif i==1: 
            group.append(3)
        elif i>=1/2:
            group.append(2)
        elif i<1/2:
            group.append(1)
        else:
            print('invalid')
    result=pd.DataFrame({})
    result['game_session']=df.game_session
    result['group']=group
    return result

In [29]:
sample1=main(sample)

Starting to process the information .......
Finish step 1, time elapsed:  00:00:06
Finish step 2, time elapsed:  00:00:09
Finish step 3, time elapsed:  00:00:09
Finish step 4, time elapsed:  00:00:10
Finish step 5, time elapsed:  00:00:11


In [30]:
check_labels_in_train_labels(sample1)

Num correct labels are: 187
Percentage of correctness: 100.0


# III) Prepare cooked data for prediction and export visualizable data

In [31]:
%%time
df_train=main(train)

Starting to process the information .......
Finish step 1, time elapsed:  00:08:27
Finish step 2, time elapsed:  00:09:43
Finish step 3, time elapsed:  00:09:45
Finish step 4, time elapsed:  00:10:25
Finish step 5, time elapsed:  00:10:27
CPU times: user 9min 45s, sys: 17.6 s, total: 10min 3s
Wall time: 10min 27s


In [32]:
%%time
df_test=main(test)

Starting to process the information .......
Finish step 1, time elapsed:  00:01:42
Finish step 2, time elapsed:  00:01:54
Finish step 3, time elapsed:  00:01:54
Finish step 4, time elapsed:  00:02:00
Finish step 5, time elapsed:  00:02:01
CPU times: user 1min 57s, sys: 4.26 s, total: 2min 1s
Wall time: 2min 1s


# Output

In [35]:
y=label(df_train).group
y=pd.DataFrame(y, columns=['group'])
df_test=groupby_iid(df_test)

In [36]:
# df_train.to_csv('output/X_train.csv',index=False)
# y.to_csv('output/label.csv',index=False)
# df_test.to_csv('output/X_test.csv',index=False)

In [37]:
print(df_train.shape,len(y), df_test.shape)

(18837, 540) 18837 (1000, 542)


In [5]:
X_train=pd.read_csv('output/X_train.csv')

In [6]:
X_train.head()

,ASM_GT,ASM__Bird Measurer (Assessment),ASM__Cart Balancer (Assessment),ASM__Cauldron Filler (Assessment),ASM__Chest Sorter (Assessment),ASM__Mushroom Sorter (Assessment),ASM_accuracy,ASM_correct,ASM_incorrect,ASM_num_assess,Assessment_title,Average_GT_assess,Average_GT_per_session,GT_title_12 Monkeys,GT_title_Air Show,GT_title_All Star Sorting,GT_title_Balancing Act,GT_title_Bird Measurer (Assessment),GT_title_Bottle Filler (Activity),GT_title_Bubble Bath,GT_title_Bug Measurer (Activity),GT_title_Cart Balancer (Assessment),GT_title_Cauldron Filler (Assessment),GT_title_Chest Sorter (Assessment),GT_title_Chicken Balancer (Activity),GT_title_Chow Time,GT_title_Costume Box,GT_title_Crystal Caves - Level 1,GT_title_Crystal Caves - Level 2,GT_title_Crystal Caves - Level 3,GT_title_Crystals Rule,GT_title_Dino Dive,GT_title_Dino Drink,GT_title_Egg Dropper (Activity),GT_title_Fireworks (Activity),GT_title_Flower Waterer (Activity),GT_title_Happy Camel,"GT_title_Heavy, Heavier, Heaviest",GT_title_Honey Cake,GT_title_Leaf Leader,GT_title_Lifting Heavy Things,GT_title_Magma Peak - Level 1,GT_title_Magma Peak - Level 2,GT_title_Mushroom Sorter (Assessment),GT_title_Ordering Spheres,GT_title_Pan Balance,GT_title_Pirate's Tale,GT_title_Rulers,GT_title_Sandcastle Builder (Activity),GT_title_Scrub-A-Dub,GT_title_Slop Problem,GT_title_Treasure Map,GT_title_Tree Top City - Level 1,GT_title_Tree Top City - Level 2,GT_title_Tree Top City - Level 3,GT_title_Watering Hole (Activity),GT_type_Activity,GT_type_Assessment,GT_type_Clip,GT_type_Game,GT_world_CRYSTALCAVES,GT_world_MAGMAPEAK,GT_world_TREETOPCITY,Record__Bird Measurer (Assessment),Record__Cart Balancer (Assessment),Record__Cauldron Filler (Assessment),Record__Chest Sorter (Assessment),Record__Mushroom Sorter (Assessment),TITLE_12 Monkeys,TITLE_Air Show,TITLE_All Star Sorting,TITLE_Balancing Act,TITLE_Bottle Filler (Activity),TITLE_Bubble Bath,TITLE_Bug Measurer (Activity),TITLE_Chicken Balancer (Activity),TITLE_Chow Time,TITLE_Costume Box,TITLE_Crystal Caves - Level 1,TITLE_Crystal Caves - Level 2,TITLE_Crystal Caves - Level 3,TITLE_Crystals Rule,TITLE_Dino Dive,TITLE_Dino Drink,TITLE_Egg Dropper (Activity),TITLE_Fireworks (Activity),TITLE_Flower Waterer (Activity),TITLE_Happy Camel,"TITLE_Heavy, Heavier, Heaviest",TITLE_Honey Cake,TITLE_Leaf Leader,TITLE_Lifting Heavy Things,TITLE_Magma Peak - Level 1,TITLE_Magma Peak - Level 2,TITLE_Ordering Spheres,TITLE_Pan Balance,TITLE_Pirate's Tale,TITLE_Rulers,TITLE_Sandcastle Builder (Activity),TITLE_Scrub-A-Dub,TITLE_Slop Problem,TITLE_Treasure Map,TITLE_Tree Top City - Level 1,TITLE_Tree Top City - Level 2,TITLE_Tree Top City - Level 3,TITLE_Watering Hole (Activity),WORLD_Leaf Leader,acc_accuracy_group,acc_num_correct,acc_num_incorrect,event_count,first_timestamp,game_session,game_time,iid_Label,installation_id,num_game_session,time_distance_assessment,2000,3010,4070,3110,4030,4021,4020,4090,4010,2080,2081,2020,2040,2030,3021,3121,2050,3020,3120,2083,2025,4035,4025,4022,2035,4040,4100,2010,2060,2070,4110,5000,5010,4031,4045,4095,4220,2075,4230,4235,4050,4080,27253bdc,77261ab5,b2dba42b,1325467d,1bb5fbdb,5e812b27,9ee1c98c,84538528,37937459,6d90d394,7040c096,5a848010,c1cac9a2,26fd2d99,dcaede90,f71c4741,f7e47413,5c3d2b2f,08fd73f3,73757a5e,2b9272f4,37c53127,d88e8f25,ac92046e,cf82af56,4a09ace1,b7dc8128,4b5efe37,2c4e6db0,363d3849,587b5989,9e4c8c7b,1cc7cfca,2dc29e21,6043a2b4,c277e121,b120f2ac,d45ed6a1,ca11f653,daac11b0,1f19558b,d02b7a8e,4901243f,beb0a7b9,02a42007,e694a35b,b88f38da,884228c8,9b01374f,56cd3b43,a44b10dc,bbfe0445,5d042115,de26c3a6,598f4598,fcfdffb6,3bfd1a65,db02c830,a1e4395d,a52b92d5,28ed704e,9d29771f,c74f40cd,7da34a02,83c6c409,3dfd4aa4,fbaf3456,5f0eb72c,c7128948,25fa8af4,6c930e6e,a5be6304,15ba1109,65abac75,bcceccc6,06372577,f28c589a,a1bbe385,d88ca108,dcb55a27,1575e76c,28f975ea,14de4c5d,9b4001e4,58a0de5c,f5b8c21a,7423acbc,e04fb33d,48349b14,cc5087a3,5154fc30,3babcb9b,7cf1bc53,5e3ea25a,86c924c4,3ddc79c3,e720d930,3323d7e9,44cb4907,8b757ab8,c7f7

In [7]:
train.head()

NameError: name 'train' is not defined